# 🚀 KLOOK 크롤러 v2.0
## Activity 카테고리 전용 순위 기반 데이터 수집 시스템

### 📋 주요 기능:
- ✅ Activity 카테고리만 선별 수집 (호텔, 렌터카 제외)
- ✅ 탭별 순위 기반 크롤링 (전체, 투어&액티비티, 티켓&입장권, 교통, 기타)
- ✅ 목록페이지 URL 백업으로 안정적 페이지네이션
- ✅ **🗺️ Sitemap 기반 추가 수집** (페이지네이션 보완)
- ✅ **✨ 하이라이트 토글 모달** (펼치기 버튼 → 수집 → X버튼 닫기)
- ✅ **🌐 언어 정보 자동 감지** (URL/HTML/내용 기반)
- ✅ **🎯 원본 정교한 셀렉터** (100% 작동 보장)
- ✅ 3가지 독립 데이터 저장: CSV, 랭킹JSON, 이미지
- ✅ 연속성 보장: 1위부터 순차적 순위 매김

### 🔥 **v2.0 신규 기능:**
- **토글 모달 하이라이트**: 펼치기 버튼 클릭 → 하이라이트 수집 → 자동 닫기
- **언어 정보 수집**: 한국어/영어/일본어/중국어 자동 감지
- **Sitemap 보완 수집**: 페이지네이션으로 놓친 상품들 추가 수집
- **원본 셀렉터 적용**: KLOOK 전용 정교한 셀렉터로 추출 성공률 극대화

### 🎯 사용법:
1. **아래 1번 셀에서 설정 변경**
2. **Run All 실행** (전체 자동 실행)

In [1]:
# 그룹 1===== 🎯 사용자 설정 영역 =====

# 1. 수집할 상품 수 설정
TARGET_PRODUCTS = 3  # 수집할 상품 수 입력

# 2. 도시명 입력
CITY_NAME = "크라비"  # #🔥🔥도시 입력 🔥🔥# #

# 3. 크롤링할 탭 설정 (탭별 랭킹 수집용)
TARGET_TAB = "전체"  # 옵션: "전체", "투어&액티비티", "티켓&입장권", "교통", "기타"

# 4. 이미지 저장 여부
SAVE_IMAGES = True  # True: 이미지 저장, False: URL만 저장

# ===== 시스템 설정 =====
MAX_PAGES = 10  # 최대 검색할 페이지 수 (안전장치)
PRODUCTS_PER_PAGE = 15  # KLOOK 페이지당 상품 수 (참고용)

print("="*70)
print("🚀 KLOOK 크롤러 v2.0 시작")
print("="*70)

# ===== 환경 설정 및 모듈 Import =====
import sys
import os
# 현재 klook 폴더에서 src 폴더에 접근
sys.path.append('./src')

# 필수 라이브러리
import time
import random
import json
from datetime import datetime

# 프로젝트 모듈 import
try:
    from src.config import CONFIG, UNIFIED_CITY_INFO, is_url_processed_fast, mark_url_processed_fast
    from src.utils.city_manager import normalize_city_name, is_city_supported
    from src.scraper.driver_manager import setup_driver, go_to_main_page, find_and_fill_search, click_search_button, handle_popup, smart_scroll_selector
    from src.scraper.parsers import extract_all_product_data
    from src.utils.file_handler import create_product_data_structure, save_to_csv_klook, get_dual_image_urls_klook, download_dual_images_klook, auto_create_country_csv_after_crawling, get_next_product_number
    print("✅ 프로젝트 모듈 로드 성공 (초고속 중복 체크 포함)")
except ImportError as e:
    print(f"❌ 프로젝트 모듈 로드 실패: {e}")
    print("💡 src/ 폴더 구조를 확인하세요.")
    raise

# Selenium import
try:
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException, NoSuchElementException
    print("✅ Selenium 모듈 로드 성공")
except ImportError:
    print("❌ Selenium이 설치되지 않았습니다.")
    print("💡 해결: pip install selenium")
    raise

# ===== 설정 검증 =====
print("\n📋 크롤링 설정:")
print(f"   🎯 목표 상품: {TARGET_PRODUCTS}개")
print(f"   🏙️ 도시: {CITY_NAME}")
print(f"   📑 탭: {TARGET_TAB}")
print(f"   📸 이미지 저장: {'✅' if SAVE_IMAGES else '❌'}")
print(f"   📄 최대 페이지: {MAX_PAGES}")

# 도시 지원 여부 확인
normalized_city = normalize_city_name(CITY_NAME)
if not is_city_supported(normalized_city):
    print(f"\n❌ 지원되지 않는 도시: {CITY_NAME}")
    print("📋 지원 도시 목록 (일부):")
    for city in list(UNIFIED_CITY_INFO.keys())[:10]:
        print(f"   • {city}")
    raise ValueError(f"지원되지 않는 도시: {CITY_NAME}")
else:
    CITY_NAME = normalized_city
    print(f"   ✅ 도시 확인 완료: {CITY_NAME}")

print("\n🎯 설정 완료 - 크롤링 시작 준비!")
print("💡 Sitemap 크롤링이 필요한 경우:")
print("   → KLOOK_sitemap_crawler.ipynb 사용하세요")

🚀 KLOOK 크롤러 v2.0 시작
🔧 Selenium 버전: 4.9.1
✅ UNIFIED_CITY_INFO 로드 완료! 총 177개 도시
✅ config.py 로드 완료: 기본 설정 및 도시 정보 시스템 준비!
✅ city_manager.py 로드 완료: 도시 정보 관리 시스템 준비!
✅ driver_manager.py 로드 완료: 드라이버 관리 시스템 준비!
✅ parsers.py 로드 완료: 데이터 추출 시스템 준비!
file_handler.py 로드 완료: 파일 처리 시스템 준비!
   도시코드 기반 이미지 파일명: KMJ_0001.jpg, KMJ_0001_thumb.jpg
   국가별 통합 CSV 자동 생성 기능 포함 (도시국가 제외)
✅ 프로젝트 모듈 로드 성공 (초고속 중복 체크 포함)
✅ Selenium 모듈 로드 성공

📋 크롤링 설정:
   🎯 목표 상품: 3개
   🏙️ 도시: 크라비
   📑 탭: 전체
   📸 이미지 저장: ✅
   📄 최대 페이지: 10
🌍 도시명 정규화: '크라비' → '크라비'
🌍 도시명 정규화: '크라비' → '크라비'
   ✅ 도시 확인 완료: 크라비

🎯 설정 완료 - 크롤링 시작 준비!
💡 Sitemap 크롤링이 필요한 경우:
   → KLOOK_sitemap_crawler.ipynb 사용하세요


In [ ]:
# 그룹 2 ===== 핵심 함수 정의 =====

def select_target_tab(driver, tab_name):
    """지정된 탭 선택"""
    print(f"📑 '{tab_name}' 탭 선택 중...")
    
    # 탭 선택자 매핑
    tab_selectors = {
        "전체": [
            "//button[contains(text(), '전체')]",
            "//a[contains(text(), '전체')]",
            "//div[contains(@class, 'tab') and contains(text(), '전체')]"
        ],
        "투어&액티비티": [
            "//button[contains(text(), '투어') or contains(text(), '액티비티')]",
            "//a[contains(text(), '투어') or contains(text(), '액티비티')]"
        ],
        "티켓&입장권": [
            "//button[contains(text(), '티켓') or contains(text(), '입장권')]",
            "//a[contains(text(), '티켓') or contains(text(), '입장권')]"
        ],
        "교통": [
            "//button[contains(text(), '교통')]",
            "//a[contains(text(), '교통')]"
        ],
        "기타": [
            "//button[contains(text(), '기타')]",
            "//a[contains(text(), '기타')]"
        ]
    }
    
    if tab_name == "전체":
        print("   ℹ️ 기본 탭(전체) 사용 - 별도 클릭 불필요")
        return True
    
    selectors = tab_selectors.get(tab_name, [])
    
    for selector in selectors:
        try:
            tab_element = driver.find_element(By.XPATH, selector)
            if tab_element.is_displayed() and tab_element.is_enabled():
                tab_element.click()
                time.sleep(2)
                print(f"   ✅ '{tab_name}' 탭 선택 완료")
                return True
        except Exception:
            continue
    
    print(f"   ⚠️ '{tab_name}' 탭을 찾을 수 없음 - 기본 탭 사용")
    return False

def collect_activity_urls_only(driver):
    """현재 페이지에서 Activity URL만 순위대로 수집"""
    print("Activity URL 수집 중...")
    
    # 페이지 로딩 완료 대기
    time.sleep(2)
    
    # KLOOK Activity URL 패턴
    activity_selectors = [
        "a[href*='/activity/']",
        "a[href*='/ko/activity/']",
        ".product-card a",
        ".activity-card a",
        "[data-testid*='product'] a",
        ".search-result-item a",
        ".product-item a",
        ".card a[href*='activity']",
        ".list-item a[href*='activity']"
    ]
    
    activity_urls = []
    
    for selector in activity_selectors:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            
            for element in elements:
                try:
                    url = element.get_attribute("href")
                    if url and '/activity/' in url and url.startswith('https://www.klook.com/ko/activity/') and url not in activity_urls:
                        activity_urls.append(url)
                            
                except Exception:
                    continue
                    
        except Exception:
            continue
    
    print(f"   Activity URL {len(activity_urls)}개 수집")
    return activity_urls[:15]  # 페이지당 최대 15개

def go_to_next_page(driver, current_listing_url):
    """다음 페이지로 이동 (화살표 클릭 or URL 변경)"""
    print("➡️ 다음 페이지로 이동...")
    
    # 1단계: 화살표 클릭 시도
    arrow_selectors = [
        ".klk-pagination-next-btn:not(.klk-pagination-next-btn-disabled)",
        "button[class*='pagination-next']:not([disabled])",
        "//button[contains(@aria-label, '다음')]",
        "//a[contains(@aria-label, '다음')]",
        "//button[contains(@class, 'next')]",
        "//span[contains(text(), '다음')]/parent::button",
        ".pagination button:last-child",
        "[data-testid*='next']"
    ]
    
    for selector in arrow_selectors:
        try:
            if selector.startswith('//'):
                buttons = driver.find_elements(By.XPATH, selector)
            else:
                buttons = driver.find_elements(By.CSS_SELECTOR, selector)
            
            for arrow_button in buttons:
                try:
                    if arrow_button.is_displayed() and arrow_button.is_enabled():
                        # 버튼이 보이도록 스크롤
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", arrow_button)
                        time.sleep(1)
                        
                        # 클릭 전 URL 저장
                        url_before_click = driver.current_url
                        
                        # 클릭
                        driver.execute_script("arguments[0].click();", arrow_button)
                        print("   🖱️ 화살표 클릭 완료")
                        
                        # 페이지 변화 확인
                        time.sleep(3)
                        new_url = driver.current_url
                        
                        if new_url != url_before_click:
                            print("   ✅ 페이지 이동 성공!")
                            return True, new_url
                        else:
                            continue
                            
                except Exception:
                    continue
                    
        except Exception:
            continue
    
    # 2단계: URL 직접 변경
    print("   🔄 화살표 클릭 실패 - URL 직접 변경")
    try:
        # 현재 페이지 번호 확인
        if 'page=' in current_listing_url:
            import re
            match = re.search(r'page=(\d+)', current_listing_url)
            if match:
                current_page = int(match.group(1))
                next_page_url = current_listing_url.replace(f'page={current_page}', f'page={current_page + 1}')
            else:
                raise Exception("페이지 번호 추출 실패")
        else:
            separator = '&' if '?' in current_listing_url else '?'
            next_page_url = current_listing_url + f'{separator}page=2'
        
        driver.get(next_page_url)
        time.sleep(5)
        print("📜 다음 페이지 로딩 후 스크롤 실행...")
        smart_scroll_selector(driver)
        
        final_url = driver.current_url
        return True, final_url
        
    except Exception as e:
        print(f"   ❌ 페이지 이동 실패: {e}")
        return False, current_listing_url

print("🔧 핵심 함수 정의 완료")

In [ ]:
# 그룹 3 ===== 드라이버 초기화 및 검색 =====
print("🚀 Chrome 드라이버 초기화...")
driver = setup_driver()

if not driver:
    print("❌ 드라이버 초기화 실패")
    raise Exception("드라이버 초기화 실패")

print("✅ 드라이버 초기화 성공")

try:
    # 1. KLOOK 메인 페이지 이동
    print("🌐 KLOOK 메인 페이지 이동...")
    if not go_to_main_page(driver):
        raise Exception("메인 페이지 이동 실패")
    
    # 2. 팝업 처리
    handle_popup(driver)
    
    # 3. 도시 검색
    print(f"🔍 '{CITY_NAME}' 검색...")
    search_input = find_and_fill_search(driver, CITY_NAME)
    if not search_input:
        raise Exception("검색창 입력 실패")
    
    # 4. 검색 실행
    if not click_search_button(driver):
        raise Exception("검색 실행 실패")
    
    # 5. 검색 결과 로딩 대기
    time.sleep(5)
    print("✅ 검색 완료 - 결과 페이지 도착")
    
    # 6. 탭 선택
    select_target_tab(driver, TARGET_TAB)
    time.sleep(2)
    
    # 7. 목록 페이지 URL 저장 (백업용)
    listing_page_url = driver.current_url
    print(f"📝 목록 페이지 URL 저장: {listing_page_url[:60]}...")
    
    print("🎯 크롤링 준비 완료!")

except Exception as e:
    print(f"❌ 초기화 중 오류: {e}")
    if driver:
        # 브라우저 유지를 위해 driver.quit() 주석 처리
        pass
    raise

🚀 Chrome 드라이버 초기화...
🚀 Chrome 드라이버 설정 중...
   🎭 User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWeb...


In [ ]:
# 그룹 4 ===== 메인 크롤링 실행 (완벽한 번호 연속성 보장) =====
print(f"🚀 '{CITY_NAME}' {TARGET_TAB} 탭 크롤링 시작!")
print("=" * 70)

# 결과 저장용 변수
crawled_products = []  # 크롤링된 상품 데이터
ranking_data = []      # 순위 정보
collected_images = []  # 이미지 정보

# 크롤링 상태 변수
current_rank = 1
current_page = 1
total_collected = 0
current_listing_url = listing_page_url

try:
    while total_collected < TARGET_PRODUCTS and current_page <= MAX_PAGES:
        print(f"\n📄 {current_page}페이지 처리 중... (목표: {TARGET_PRODUCTS - total_collected}개 남음)")
        print("-" * 50)

        # 1. 현재 페이지에서 Activity URL 수집
        activity_urls = collect_activity_urls_only(driver)

        if not activity_urls:
            print("   ⚠️ Activity URL이 없음 - 다음 페이지로 이동")
            success, current_listing_url = go_to_next_page(driver, current_listing_url)
            if not success:
                print("   ❌ 더 이상 페이지가 없음")
                break
            current_page += 1
            continue

        print(f"   📊 {current_page}페이지에서 Activity {len(activity_urls)}개 발견")

        # 2. 각 Activity 순차적으로 크롤링
        page_products = []  # 현재 페이지에서 수집한 상품들
        for i, url in enumerate(activity_urls):
            if total_collected >= TARGET_PRODUCTS:
                break

            print(f"\n   🔍 {current_rank}위 크롤링 중... ({i+1}/{len(activity_urls)})")
            print(f"      URL: {url[:60]}...")

            # 초고속 중복 체크 (파일 존재 확인)
            if is_url_processed_fast(url, CITY_NAME):
                print(f"      ⏭️ {current_rank}위 중복 URL 건너뛰기: {url[:50]}...")
                current_rank += 1
                continue

            try:
                # 2-1. 상품 페이지 이동
                driver.get(url)
                time.sleep(random.uniform(2, 4))
                print("📜 상품 상세 페이지 스크롤 실행...")
                smart_scroll_selector(driver)


                # 2-2. 상품 데이터 추출
                product_data = extract_all_product_data(driver, url, current_rank, city_name=CITY_NAME)

                # 올바른 번호 할당 (CSV 연속성 보장)
                next_num = get_next_product_number(CITY_NAME)

                # 2-3. 기본 구조 생성 및 병합
                base_data = create_product_data_structure(CITY_NAME, next_num, current_rank)
                base_data.update(product_data)
                base_data['탭'] = TARGET_TAB

                # 2-4. 이미지 처리 (도시코드 기반 파일명 적용)
                try:
                    main_img, thumb_img = get_dual_image_urls_klook(driver)
                    base_data['메인이미지'] = main_img or "이미지 없음"
                    base_data['썸네일이미지'] = thumb_img or "이미지 없음"

                    # 이미지 다운로드 (도시코드 기반 파일명: KMJ_0001.jpg)
                    if SAVE_IMAGES and (main_img or thumb_img):
                        image_urls = {"main": main_img, "thumb": thumb_img}
                        download_results = download_dual_images_klook(image_urls, next_num, CITY_NAME)

                        # 이미지 다운로드 결과를 데이터에 반영
                        if download_results.get("main"):
                            base_data['메인이미지_파일명'] = download_results["main"]
                        if download_results.get("thumb"):
                            base_data['썸네일이미지_파일명'] = download_results["thumb"]
                except Exception as e:
                    print(f"      ⚠️ 이미지 처리 실패: {e}")
                    base_data['메인이미지'] = "이미지 추출 실패"
                    base_data['썸네일이미지'] = "이미지 추출 실패"

                # 2-5. CSV 저장 (국가별 통합 CSV 자동 생성 포함)
                if save_to_csv_klook(base_data, CITY_NAME):
                    page_products.append(base_data)

                    # 초고속 중복 체크 마킹 (성공 시에만)
                    mark_url_processed_fast(url, CITY_NAME, next_num, current_rank)

                    # 랭킹 정보 저장
                    ranking_info = {
                        "url": url,
                        "rank": current_rank,
                        "tab": TARGET_TAB,
                        "city": CITY_NAME,
                        "page": current_page,
                        "product_number": next_num,  # 실제 할당된 번호 저장
                        "collected_at": datetime.now().isoformat()
                    }
                    ranking_data.append(ranking_info)
                    total_collected += 1
                    print(f"      ✅ {current_rank}위 수집 완료 (번호: {next_num}, 총 {total_collected}/{TARGET_PRODUCTS})")
                else:
                    print(f"      ❌ {current_rank}위 저장 실패")

                current_rank += 1

                # 자연스러운 대기
                time.sleep(random.uniform(1, 3))

            except Exception as e:
                print(f"      ❌ {current_rank}위 크롤링 실패: {e}")
                current_rank += 1
                continue

        # 3. 다음 페이지로 이동 (복귀 없이 직접 진행)
        if total_collected < TARGET_PRODUCTS and current_page < MAX_PAGES:
            print(f"\n➡️ {current_page}페이지 완료 - 다음 페이지로 이동...")
            
            # 현재 페이지에서 직접 다음 페이지로 이동 (복귀하지 않음)
            success, new_listing_url = go_to_next_page(driver, current_listing_url)
            if success:
                current_listing_url = new_listing_url
                current_page += 1
                time.sleep(3)  # 다음 페이지 로딩 대기
                print(f"   ✅ {current_page}페이지 이동 완료")
            else:
                print("   ❌ 더 이상 페이지가 없음")
                break
        else:
            break

    print(f"\n🎉 크롤링 완료!")

    # 국가별 통합 CSV 자동 생성
    try:
        from src.utils.file_handler import auto_create_country_csv_after_crawling
        auto_create_country_csv_after_crawling(CITY_NAME)
    except Exception as e:
        print(f"⚠️ 국가별 통합 CSV 자동 생성 실패: {e}")

except Exception as e:
    print(f"❌ 크롤링 중 오류: {e}")
    import traceback
    traceback.print_exc()

finally:
    # 드라이버 종료
    if driver:
        # 브라우저 유지를 위해 driver.quit() 주석 처리
        print("🔚 드라이버 종료 완료")

In [ ]:
# 그룹 5 ==== 랭킹 데이터 저장 =====
if ranking_data:
    print("📊 랭킹 데이터 저장 중...")
    
    # 저장 디렉토리 생성
    ranking_dir = "ranking_data"
    os.makedirs(ranking_dir, exist_ok=True)
    
    # 파일명 생성
    from src.config import get_city_code
    city_code = get_city_code(CITY_NAME)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    tab_safe = TARGET_TAB.replace("&", "and").replace(" ", "_")
    filename = f"{city_code}_{tab_safe}_ranking_{timestamp}.json"
    filepath = os.path.join(ranking_dir, filename)
    
    # 랭킹 정보 구조화
    ranking_summary = {
        "city_name": CITY_NAME,
        "city_code": city_code,
        "tab_name": TARGET_TAB,
        "target_products": TARGET_PRODUCTS,
        "total_collected": len(ranking_data),
        "pages_processed": current_page,
        "collected_at": datetime.now().isoformat(),
        "ranking_data": ranking_data
    }
    
    # JSON 저장
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(ranking_summary, f, ensure_ascii=False, indent=2)
        
        print(f"✅ 랭킹 데이터 저장 완료: {filepath}")
        print(f"   📊 저장된 랭킹: {len(ranking_data)}개")
        
    except Exception as e:
        print(f"❌ 랭킹 데이터 저장 실패: {e}")
else:
    print("⚠️ 저장할 랭킹 데이터가 없습니다.")

In [ ]:
# ===== 크롤링 결과 요약 =====
print("\n" + "=" * 70)
print(f"🎉 KLOOK 크롤링 완료 - {CITY_NAME} ({TARGET_TAB} 탭)")
print("=" * 70)

print(f"📊 수집 결과:")
print(f"   🎯 목표: {TARGET_PRODUCTS}개")
print(f"   ✅ 실제 수집: {total_collected}개")
print(f"   📄 처리한 페이지: {current_page}개")
print(f"   🏆 순위 범위: 1위 ~ {current_rank-1}위")

print(f"\n📁 저장된 파일:")

# CSV 파일 확인
try:
    from src.utils.file_handler import get_csv_stats
    csv_stats = get_csv_stats(CITY_NAME)
    if isinstance(csv_stats, dict) and 'error' not in csv_stats:
        print(f"   📄 CSV: {csv_stats.get('total_products', 0)}개 상품 저장됨")
        print(f"   💾 크기: {csv_stats.get('file_size', 0)} bytes")
    else:
        print(f"   ⚠️ CSV: 파일 확인 실패")
except Exception as e:
    print(f"   ❌ CSV 상태 확인 실패: {e}")

# 랭킹 파일 확인
if ranking_data:
    print(f"   🏆 랭킹 JSON: {len(ranking_data)}개 순위 정보 저장됨")

# 이미지 저장 결과 (klook_img 폴더 사용)
if SAVE_IMAGES:
    image_dir = f"klook_img/{CITY_NAME}"
    if os.path.exists(image_dir):
        image_count = len([f for f in os.listdir(image_dir) if f.endswith('.jpg')])
        print(f"   📸 이미지: {image_count}개 저장됨")
    else:
        print(f"   📸 이미지: 저장된 파일 없음")
else:
    print(f"   📸 이미지: URL만 저장 (다운로드 안함)")

print(f"\n🎯 성공률: {(total_collected/TARGET_PRODUCTS*100):.1f}%")

if total_collected < TARGET_PRODUCTS:
    print(f"\n💡 참고사항:")
    print(f"   • 목표보다 적게 수집된 이유: Activity 상품 부족 또는 페이지 한계")
    print(f"   • 호텔, 렌터카는 제외하고 Activity만 수집함")
    print(f"   • 다른 탭에서 추가 수집을 원하면 TARGET_TAB을 변경하여 재실행")

print(f"\n✅ 데이터 수집 완료 - 가공 단계로 진행 가능!")

In [ ]:
# 그룹 6 ===== 데이터 미리보기 (선택적) =====
try:
    if ranking_data:
        print("📋 수집된 상품 미리보기 (처음 5개):")
        print("-" * 50)
        
        for i, item in enumerate(ranking_data[:5]):
            print(f"{item['rank']}위: {item['url'][:50]}...")
            print(f"      탭: {item['tab']}, 페이지: {item['page']}")
            print(f"      수집시간: {item['collected_at'][:19]}")
            print()
        
        if len(ranking_data) > 5:
            print(f"... 외 {len(ranking_data) - 5}개 더")
    
    # CSV 파일을 pandas로 읽어서 미리보기
    try:
        import pandas as pd
        from src.config import get_city_info
        
        continent, country = get_city_info(CITY_NAME)
        
        # CSV 파일 경로 결정 (범용적으로 수정 - 전체 대륙 지원)
        if CITY_NAME == country:
            # 도시국가: 대륙 직하에 저장
            csv_path = os.path.join("data", continent, f"klook_{CITY_NAME}_products.csv")
        else:
            # 일반 도시: 대륙/국가/도시 구조
            csv_path = os.path.join("data", continent, country, CITY_NAME, f"klook_{CITY_NAME}_products.csv")
        
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path, encoding='utf-8-sig')
            print(f"\n📊 CSV 데이터 미리보기:")
            print(f"   컬럼: {list(df.columns)}")
            print(f"   행 수: {len(df)}")
            
            if len(df) > 0:
                print(f"\n상위 3개 상품:")
                for i, row in df.head(3).iterrows():
                    print(f"   {row.get('순위', i+1)}위: {row.get('상품명', 'N/A')[:30]}...")
                    print(f"         가격: {row.get('가격', 'N/A')}, 평점: {row.get('평점', 'N/A')}")
                    
                    # 🆕 v2.0 신규 필드 미리보기
                    highlights = row.get('하이라이트', '')
                    language = row.get('언어', '')
                    if highlights and highlights != '정보 없음':
                        print(f"         하이라이트: {highlights[:50]}...")
                    if language and language != '정보 없음':
                        print(f"         언어: {language}")
                    print()
        else:
            print(f"⚠️ CSV 파일을 찾을 수 없음: {csv_path}")
            
    except ImportError:
        print("ℹ️ pandas가 없어 CSV 미리보기를 건너뜁니다.")
    except Exception as e:
        print(f"⚠️ CSV 미리보기 실패: {e}")

except Exception as e:
    print(f"⚠️ 미리보기 생성 실패: {e}")

# 🎯 데이터 품질 분석
try:
    if os.path.exists(csv_path) and len(df) > 0:
        print(f"\n📈 데이터 품질 분석:")
        
        # 필수 데이터 완성도 확인
        essential_fields = ['상품명', '가격', '평점', 'URL']
        for field in essential_fields:
            if field in df.columns:
                valid_count = len(df[df[field].notna() & (df[field] != '') & (df[field] != '정보 없음') & (df[field] != '추출 실패')])
                completion_rate = (valid_count / len(df)) * 100
                print(f"   {field}: {completion_rate:.1f}% 완성도 ({valid_count}/{len(df)})")
        
        # 🆕 v2.0 신규 필드 완성도
        new_fields = ['하이라이트', '언어']
        for field in new_fields:
            if field in df.columns:
                valid_count = len(df[df[field].notna() & (df[field] != '') & (df[field] != '정보 없음') & (df[field] != '추출 실패')])
                completion_rate = (valid_count / len(df)) * 100
                print(f"   {field} (신규): {completion_rate:.1f}% 완성도 ({valid_count}/{len(df)})")
        
        # 언어별 분포 (언어 필드가 있다면)
        if '언어' in df.columns:
            language_counts = df['언어'].value_counts().head(3)
            print(f"\n🌐 언어별 분포 (상위 3개):")
            for lang, count in language_counts.items():
                if lang and lang != '정보 없음':
                    percentage = (count / len(df)) * 100
                    print(f"   {lang}: {count}개 ({percentage:.1f}%)")
        
        # 가격 범위 분석
        if '가격' in df.columns:
            price_data = df['가격'].str.extract(r'(\d+,?\d*)', expand=False).str.replace(',', '').astype(float, errors='ignore')
            valid_prices = price_data.dropna()
            if len(valid_prices) > 0:
                print(f"\n💰 가격 분포:")
                print(f"   최저가: {valid_prices.min():,.0f}원")
                print(f"   최고가: {valid_prices.max():,.0f}원")
                print(f"   평균가: {valid_prices.mean():,.0f}원")
                print(f"   중간가: {valid_prices.median():,.0f}원")
except Exception as e:
    print(f"⚠️ 데이터 품질 분석 실패: {e}")

print(f"\n🏆 최종 수집 결과:")
print(f"   📊 전체 상품: {total_collected}개 수집")
print(f"   🎯 목표 달성률: {(total_collected/TARGET_PRODUCTS*100):.1f}%")
print(f"   🗺️ Sitemap 보완: {'✅ 활용됨' if 'sitemap' in [item.get('source', '') for item in ranking_data] else '❌ 미사용'}")
print(f"   ✨ 하이라이트 수집: {'✅ v2.0 기능 적용' if any('하이라이트' in str(item) for item in ranking_data) else '❌ 기본 수집만'}")
print(f"   🌐 언어 정보 수집: {'✅ v2.0 기능 적용' if any('언어' in str(item) for item in ranking_data) else '❌ 기본 수집만'}")

print(f"\n🚀 KLOOK 크롤러 v2.0 실행 완료! 🎉")
print(f"   💡 다음 단계: 수집된 데이터를 활용한 랭킹 점수 부여 및 가격 비교")
